## Registering our Model
### Getting our best runs

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import os

In [ ]:
#Get experiment
mlflow.set_tracking_uri("sqlite:///mlflow.db")
experiment = mlflow.get_experiment_by_name("elastic_net_experiment")
experiment

In [ ]:
# Get experiment runs
client = MlflowClient()
runs = client.search_runs(experiment_ids = [experiment.experiment_id])
runs

In [ ]:
#Get the best metrics value and use it to find the best run

metrics = [run.data.metrics["rmse"] for run in runs]
best_rmse = min(metrics)
best_run = [run for run in runs if run.data.metrics["rmse"] == best_rmse][0]
best_run.info

In [ ]:
# Register the model from the best run and log it

model_name = "my_enet_model"
mlflow.set_experiment(experiment_name = "elastic_net_experiment")
model_uri = f"runs:/{best_run.info.run_id}/model"
model_version = mlflow.register_model(model_uri = model_uri, name = model_name)
model_version

## Loading our model

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
model = client.get_registered_model(name = "my_enet_model")
model

In [ ]:
model_artifact = mlflow.sklearn.load_model(model_uri)
model_artifact

## Running inference

In [ ]:
from sklearn import datasets
import numpy as np

diabetes = datasets.load_diabetes()
data = diabetes.data
pred = model_artifact.predict(data)
pred

## Adding metadata to our model

In [ ]:
client.set_model_version_tag(model_name, model_version.version, key = "prediction_works", value = "true")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)
model_version_info.tags

## Promoting our model to Staging

We are happy with our model (for now), so let's promote it to Staging

In [ ]:
client.transition_model_version_stage(model_name, model_version.version, stage = "Staging")

In [ ]:
model_version_info = client.get_model_version(model_name, model_version.version)
model_version_info.current_stage